In [42]:
# import necessary libs
import torch
import pickle
import numpy as np
from dis import dis
from astyx_utils import *
import matplotlib.pyplot as plt
from PIL import Image

In [43]:
# Define constants
DATA_DIR = "./data/"
IMAGE_DIR = DATA_DIR + "image/"
PCD_DIR = DATA_DIR + "pcd/"
BOX2D_DIR = DATA_DIR + "box2d/"
BOX3D_DIR = DATA_DIR + "box3d/"
PRED_DICT_2D_DIR = DATA_DIR + "pred_dict_2D/"
PRED_DICT_3D_DIR = DATA_DIR + "pred_dict_3D/"

In [44]:
# Getting filename for image, pcd, box2d and box3d
file_list_image=os.listdir(IMAGE_DIR)
file_list_no_ex = [x.split('.')[0] for x in file_list_image]
file_list_pcd = [x+".npy" for x in file_list_no_ex]
file_list_box2d = [x+"_box2d.pickle" for x in file_list_no_ex]
file_list_box3d = [x+"_box3d.pickle" for x in file_list_no_ex]

In [45]:
# Read images from camera
image_list = []
for path in file_list_image:
    img = plt.imread(IMAGE_DIR + path)
    image_list.append(img)
images = np.array(image_list)

In [46]:
# Read pcd from lidar
pcd_list = []
for path in file_list_pcd:
    pcd = np.load(PCD_DIR + path)
    pcd_list.append(pcd)
pcds = np.array(pcd_list)

In [47]:
# Read 2d boxes from Detectron
# box2d_list = []
# for path in file_list_box2d:
#     with open(BOX2D_DIR + path, 'rb') as handle:
#         box2d = pickle.load(handle)
#         box2d_list.append(box2d.numpy())
# boxes2d = np.array(box2d_list)
# boxes2d

In [48]:
# Read 3d boxes center from OpenPCDet with pointpillar
# box3d_list = []
# for path in file_list_box3d:
#     with open(BOX3D_DIR + path, 'rb') as handle:
#         box3d = pickle.load(handle)
#         box3d_list.append(box3d.numpy()[:,:3])
# boxes3d = np.array(box3d_list)

In [49]:
# boxes3d_to_img = []
# for frame in boxes3d:
#     frame_list = lidar2CameraOurs(frame[:, [1, 2, 0]])
#     boxes3d_to_img.append(frame_list)

In [50]:
def detect_inlier(boxes2d, boxes3d):
    """
    Detect from the 3D boudning boexes if they are inside the 2D bounding boxes
    Args:
    boxes2d : np.array of shape [n, 4]
              boxes2d element [y1, x1, y2, x2]
              (x1,y1)-------
                |          |
                |          |
                |          |
                --------(x2,y2)
    boxes3d : np.array of shape [m, 2]
              boxes3d element [x, y, z]
    Return:
    list : [boolean] indicating if 3d boxes are inliers
    """
    result = []
    for box3d in boxes3d:
        inlier = False
        for box2d_raw in boxes2d:
            box2d = box2d_raw[[1,0,3,2]]
            if box3d[0] >= box2d[0] and box3d[0] <= box2d[2] and\
               box3d[1] >= box2d[1] and box3d[1] <= box2d[3]:
               inlier = True
        result.append(inlier)
    return result

In [51]:
# sum(detect_inlier(boxes2d[0], boxes3d_to_img[0]))

In [58]:
for file in file_list_no_ex:
    pred_dict_2D_path = PRED_DICT_2D_DIR + file + "_pred_dict_2D.pickle"
    pred_dict_3D_path = PRED_DICT_3D_DIR + file + "_pred_dict.pickle"
    with open(pred_dict_2D_path, 'rb') as handle:
        pred_dict_2D = pickle.load(handle)
        boxes_2D = pred_dict_2D["boxes"]
        scores_2D = pred_dict_2D["scores"]
        classes_2D = pred_dict_2D["classes"]
    with open(pred_dict_3D_path, 'rb') as handle:
        pred_dict_3D = pickle.load(handle)
        boxes_3D = pred_dict_3D["pred_boxes"]
        scores_3D = pred_dict_3D["pred_scores"]
        labels_3D = pred_dict_3D["pred_labels"]
    boxes_3D_filtered = []
    scores_3D_filtered = []
    labels_3D_filtered = []
    boxes_3D_to_img = lidar2CameraOurs(boxes_3D[:, [1, 2, 0]])
    boxes_3D_to_img = boxes_3D_to_img[:,[1,0]]
    result = detect_inlier(boxes_2D, boxes_3D_to_img)
    pred_dict_3D_filtered = {}
    for i in range(len(result)):
        if result[i]:
            if scores_3D[i] > 0.2:
                boxes_3D_filtered.append(boxes_3D[i])
                scores_3D_filtered.append(scores_3D[i])
                labels_3D_filtered.append(labels_3D[i])
        elif scores_3D[i] > 0.5:
                boxes_3D_filtered.append(boxes_3D[i])
                scores_3D_filtered.append(scores_3D[i])
                labels_3D_filtered.append(labels_3D[i])
    pred_dict_3D_filtered["pred_boxes"] = boxes_3D_filtered
    pred_dict_3D_filtered["pred_scores"] = scores_3D_filtered
    pred_dict_3D_filtered["pred_labels"] = labels_3D_filtered
    output_file_name = "./outputs/" + file + "_pred_dict_filtered.pickle"
    with open(output_file_name, 'wb') as handle:
        pickle.dump(pred_dict_3D_filtered, handle)


In [53]:
print(pred_dict_3D_filtered)

{'pred_boxes': [tensor([22.4676,  5.6409,  0.2507,  4.4106,  1.7534,  1.5599,  2.5399]), tensor([49.9992, -7.6712,  0.7731,  4.2005,  1.6797,  1.5968,  2.5414]), tensor([28.9171, -3.3993,  0.2875,  4.0055,  1.6275,  1.4736,  1.6085])], 'pred_scores': [tensor(0.4220), tensor(0.1190), tensor(0.1043)], 'pred_labels': [tensor(1), tensor(1), tensor(1)]}


In [54]:
# experiment space
from cProfile import label
import numpy as np
from torch import tensor
import open3d
import pickle
import open3d_vis_utils as V


In [55]:
data = np.load('./data/pcd/0038.npy')
img = plt.imread('./data/image/0038.png')
with open('./data/pred_dict_3D/0038_pred_dict.pickle', 'rb') as handle:
    pred_dict = pickle.load(handle)
box = pred_dict['pred_boxes']
score = pred_dict['pred_scores']
label = pred_dict['pred_labels']

with open("./data/pred_dict_2D/0038_pred_dict_2D.pickle", 'rb') as handle:
        pred_dict_2D = pickle.load(handle)
        boxes_2D = pred_dict_2D["boxes"]
        scores_2D = pred_dict_2D["scores"]
        classes_2D = pred_dict_2D["classes"]

select = 5

if select != 0:
    box = box[:select,:]
    score = score[:select]
    label = label[:select]

print(box)

boxes_3D_to_img = lidar2CameraOurs(box[:, [1, 2, 0]])
boxes_3D_to_img = boxes_3D_to_img[:,[1,0]] 
print(boxes_2D)
print(boxes_3D_to_img)
result = detect_inlier(boxes_2D, boxes_3D_to_img)
print(result)

tensor([[12.0305, 12.3317, -0.2406,  4.2214,  1.6851,  1.5094,  2.6132],
        [40.7842, -8.5878,  0.4647,  3.8367,  1.6026,  1.4607,  2.6140],
        [28.8681,  2.7534,  0.5071,  4.0162,  1.6076,  1.5282,  1.6533],
        [46.2989,  6.2338,  0.8187,  4.2639,  1.6388,  1.5250,  1.6065],
        [31.3598, -0.1179,  0.5201,  3.9506,  1.6822,  1.5831,  2.8671]])
[[ 601.7733   499.3728   672.74225  535.5947 ]
 [ 249.68852  494.4106   298.2574   526.0343 ]
 [ 904.0054   472.43103 1078.1804   561.80304]]
[[ 558.44561768 2363.00048828]
 [ 515.07202148  654.06481934]
 [ 506.54492188 1077.21374512]
 [ 506.38220215 1131.51623535]
 [ 507.89971924  940.09527588]]
[False, True, True, False, True]


In [56]:
V.draw_scenes(data, ref_boxes=box)